In [7]:
import numpy as np, math
from core.finance.entity import History
from core.finance.context import Context

SPAN = 5
SLIDE = 1
ticks = ["XOM", "WMT", "PG", "MSFT",
         "GE", "T", "JNJ", "CVX"]

def get_all_ticks(ticker_repository, history_repository):
    results = []
    for tick in ticker_repository.find_all_ticks():
        histories = history_repository.find_history_by_tick(tick)
        if len(histories) == 0:
            print("insufficient data:" + tick)
            continue
        results.append(tick)
        dates = []
        for h in histories:
            dates.append(h.date)
    return results, dates

def fill(result, id, n, histories):
    rs = compute_rs(histories)
    t = len(rs)
    m = np.mean(rs)
    sigma = np.std(rs)
    for j, r in enumerate(rs):
        result[id][j] = (r - m) / (sigma * math.sqrt(n * t))

def compute_rs(histories: [History]):
    previous = None
    results = []
    for h in histories:
        if previous is None:
            previous = h.open_price
            continue
        result = math.log(h.open_price) - math.log(previous)
        results.append(result)
        previous = h.open_price
    return results

def check_norm(coefficient):
    result = 0
    for a in coefficient.flatten():
        result = result + a * a
    if result > 1.01 or result < 0.99:
        raise ValueError("the value of norm is illegal: {}".format(result))
    print("norm check OK: {}".format(result))

In [8]:
context = Context()
ticker_repository = context.get_ticker_repository()
history_repository = context.get_history_repository()
coefficient_repository = context.get_coefficient_repository()
all_ticks, dates = get_all_ticks(ticker_repository, history_repository)
coefficient_repository.save_dates(dates)
coefficient_repository.save_tickers(all_ticks)

insufficient data:UTX


In [9]:
from_index = 0
coefficients = []
while from_index + SPAN < len(dates):
    dimension = len(ticks)
    coefficient = np.zeros((dimension, SPAN - 1))
    for id, tick in enumerate(ticks):
        histories = history_repository.find_history_by_tick_and_span(tick, SPAN, from_index)
        fill(coefficient, id, dimension, histories)
    coefficient_repository.save(coefficient, SPAN, from_index, ticks)
    coefficients.append(coefficient)
    from_index = from_index + SLIDE
    print("--------------")
    print("From:{}".format(from_index))
    print(coefficient)
    check_norm(coefficient)

../data/processed/coefficients/coefficient_5_0_XOMWMTPGMSFTGETJNJCVX.npy
--------------
From:1
[[ 0.24663215 -0.03258083  0.03475249 -0.24880381]
 [ 0.24649349 -0.14572539 -0.18809383  0.08732573]
 [-0.09094819  0.00141131 -0.19263207  0.28216895]
 [ 0.15491781  0.17902363 -0.08575323 -0.24818822]
 [-0.12416185  0.00751299 -0.16830733  0.28495619]
 [ 0.16401238  0.15120725 -0.27064653 -0.0445731 ]
 [ 0.07264085 -0.08770139 -0.22902666  0.2440872 ]
 [ 0.20281317  0.07990517 -0.00439721 -0.27832112]]
norm check OK: 0.9999999999999996
../data/processed/coefficients/coefficient_5_1_XOMWMTPGMSFTGETJNJCVX.npy
--------------
From:2
[[ 0.07545649  0.18872763 -0.28828357  0.02409945]
 [-0.12896336 -0.19818981  0.2518224   0.07533077]
 [-0.0752959  -0.25389338  0.18311337  0.14607591]
 [-0.00704897 -0.10940873 -0.17220427  0.28866197]
 [-0.06288482 -0.24829205  0.22968645  0.08149042]
 [ 0.11896486 -0.26071512 -0.05724283  0.19899308]
 [-0.09303699 -0.23177262  0.23267191  0.0921377 ]
 [ 0.16263